## *Rodar notebook toda quinta-feira já que às quintas os atletas têm suas habilidades atualizadas após o treino*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db
from datetime import datetime

In [2]:
# Mantendo o padrão de como o sokker mostra as datas nos dados que disponibiliza
agora= str(datetime.now()).split('.')[0]
print("Data:", agora)

Data: 2024-11-07 08:44:19


In [3]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


Como meu clube não pode ter valores nulos, os dos outros usuários sim (não tenho acesso a alguns dados de seus jogadores), eu insiro uma cláusula if antes de iniciar a inserção dos dados na base de dados para conferir.

In [4]:
# URL da API
url= f'https://sokker.org/xml/players-{club_id}.xml'

# Pegando os dados dos atletas
df_players= pega_dados(url, sessao, search= 'player')

# Inserindo a data de hoje que será importante para acompanharmos suas evoluções
df_players= df_players.assign(date_extracted= agora)

# Soma de valores nulos ou duplicados
nulos= df_players.isnull().sum().sum()
duplicados=  df_players.duplicated().sum()


if (nulos > 0) or (duplicados > 0):
    print('Há valores nulos ou duplicados. Confira os dados e tente novamente.')
    
else:
    print('Não há valores nulos ou duplicados. Inserindo dados na base de dados "dados.sokker.db"...')
    
    db= conecta_ao_db('dados_sokker.db')
    curs= db.cursor()
    
    df_players.to_sql('players', con=db, if_exists='append', index= False) 
    print('Dados extraídos com sucesso!')

Não há valores nulos ou duplicados. Inserindo dados na base de dados "dados.sokker.db"...
Dados extraídos com sucesso!


In [5]:
df = pd.read_sql_query("SELECT * FROM players", db)

In [6]:
df

,ID,date_extracted,name,surname,countryID,age,teamID,wage,height,weight,...,skillDiscipline,skillPace,skillTechnique,skillPassing,skillKeeper,skillDefending,skillPlaymaking,skillScoring,trainingPosition,isInTrainingSlot
0,36519432,2021-07-27 14:44:35,Krastan,Apostolov,54,29,79085,33900.0,177.0,71.5,...,5,15,3,9,10,5,5,3,None,None
1,37968035,2021-07-27 14:44:35,Romildo,Barbadiano,21,29,79085,3900.0,171.0,63.9,...,1,5,8,3,0,8,1,3,None,None
2,37968034,2021-07-27 14:44:35,Giovani,Barriguinha,21,27,79085,2600.0,172.0,76.3,...,0,6,2,5,1,5,1,4,None,None
3,36103236,2021-07-27 14:44:35,Ilario,Bartolozzi,10,29,79085,16900.0,169.0,67.3,...,9,13,7,8,0,8,9,8,None,None
4,37968028,2021-07-27 14:44:35,Cecilio,Bazzarella,21,20,79085,4800.0,175.0,77.4,...,1,7,1,9,0,5,7,7,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,39502944,2024-11-07 08:44:19,Srboljub,Pleh,47,18,79085,5100.0,177.0,71.5,...,8,11,4,6,0,2,4,5,2,true
3971,39058237,2024-11-07 08:44:19,Marqueto,Pratto,21,21,79085,10500.0,178.0,74.5,...,11,11,11,8,0,9,8,4,2,true
3972,39315725,2024-11-07 08:44:19,Đỗ,Quốc Việt,84,19,79085,8000.0,182.0,81.1,...,7,11,5,3,0,11,5,3,2,true
3973,38532631,2024-11-07 08:44:19,Constantín,Scher,19,27,79085,41200.0,170.0,65.1,...,15,12,13,11,2,9,13,10,2,false


In [7]:
len(df)

3975

## 

In [8]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()

In [9]:
df.to_csv('All_Players_Data_Sokker.csv', index=0)